In [51]:
import numpy as np
import tensorflow as tf
import datetime
import time

In [52]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [53]:
import threading # for multithread
from multiprocessing import Process # for multiprocess

In [54]:
#!pip uninstall tensorflow

In [56]:
#Processing Units logs
log_device_placement = True

#num of multiplications to perform
n = 10

In [57]:
# Example: compute A^n + B^n on 2 GPUs

# Create random large matrix
A = np.random.rand(10000, 10000).astype('float32')
B = np.random.rand(10000, 10000).astype('float32')

# Creates a graph to store results
c1 = []
c2 = []

# Define matrix power
def matpow(M, n):
    if n < 1: #Abstract cases where n < 1
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

In [58]:
# Single GPU computing

with tf.device('/gpu:0'):
    a = tf.constant(A)
    b = tf.constant(B)
    #compute A^n and B^n and store results in c1
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c1) #Addition of all elements in c1, i.e. A^n + B^n

t1_1 = datetime.datetime.now()
print(sum)
t2_1 = datetime.datetime.now()


tf.Tensor(
[[9.7583883e+36 9.6945760e+36 9.7628884e+36 ... 9.7117920e+36
  9.7154251e+36 9.8148900e+36]
 [9.7593136e+36 9.6953455e+36 9.7636585e+36 ... 9.7127319e+36
  9.7163790e+36 9.8155428e+36]
 [9.7961566e+36 9.7321745e+36 9.8007595e+36 ... 9.7493879e+36
  9.7530223e+36 9.8530450e+36]
 ...
 [9.8048077e+36 9.7406145e+36 9.8091906e+36 ... 9.7580073e+36
  9.7616854e+36 9.8613139e+36]
 [9.7585100e+36 9.6947047e+36 9.7630165e+36 ... 9.7119276e+36
  9.7155467e+36 9.8150732e+36]
 [9.7693243e+36 9.7050221e+36 9.7733592e+36 ... 9.7226868e+36
  9.7264644e+36 9.8249640e+36]], shape=(10000, 10000), dtype=float32)


In [59]:
# Multi GPU computing
# GPU:0 computes A^n
with tf.device('/gpu:0'):
    #compute A^n and store result in c2
    a = tf.constant(A)
    c2.append(matpow(a, n))

#GPU:1 computes B^n
with tf.device('/gpu:1'):
    #compute B^n and store result in c2
    b = tf.constant(B)
    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c2) #Addition of all elements in c2, i.e. A^n + B^n

t1_2 = datetime.datetime.now()
print(sum)
t2_2 = datetime.datetime.now()

tf.Tensor(
[[9.7583883e+36 9.6945760e+36 9.7628884e+36 ... 9.7117920e+36
  9.7154251e+36 9.8148900e+36]
 [9.7593136e+36 9.6953455e+36 9.7636585e+36 ... 9.7127319e+36
  9.7163790e+36 9.8155428e+36]
 [9.7961566e+36 9.7321745e+36 9.8007595e+36 ... 9.7493879e+36
  9.7530223e+36 9.8530450e+36]
 ...
 [9.8048077e+36 9.7406145e+36 9.8091906e+36 ... 9.7580073e+36
  9.7616854e+36 9.8613139e+36]
 [9.7585100e+36 9.6947047e+36 9.7630165e+36 ... 9.7119276e+36
  9.7155467e+36 9.8150732e+36]
 [9.7693243e+36 9.7050221e+36 9.7733592e+36 ... 9.7226868e+36
  9.7264644e+36 9.8249640e+36]], shape=(10000, 10000), dtype=float32)


In [60]:
print ("Single GPU computation time: " , str(t2_1-t1_1))
print ("Multi GPU computation time: " , str(t2_2-t1_2))

Single GPU computation time:  0:00:00.001210
Multi GPU computation time:  0:00:00.000944
